In [2]:
import pickle
import numpy
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
dataSet  = pd.read_csv("troop_movements10m.csv")
display(dataSet)

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst
...,...,...,...,...,...,...,...,...
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee


In [4]:
#checking how many null values for location x and y
dataSet.isnull().sum(axis = 0)

timestamp          0
unit_id            0
unit_type          0
location_x       500
location_y       500
destination_x      0
destination_y      0
homeworld          0
dtype: int64

In [5]:
#filling these null values and replacing them with the mean
dataSet.location_x.fillna(dataSet.location_x.mean(), inplace=True)
dataSet.location_y.fillna(dataSet.location_y.mean(), inplace=True)
dataSet.isnull().sum(axis = 0)

timestamp        0
unit_id          0
unit_type        0
location_x       0
location_y       0
destination_x    0
destination_y    0
homeworld        0
dtype: int64

In [6]:
#replace invalid unit with unknown 
dataSet = dataSet.replace("invalid_unit","unknown")

In [7]:
import pyarrow.parquet as pq
import pyarrow as pa
dataSet.to_parquet("cleaned_troop_movements10m.parquet")


In [17]:
newDataSet = pd.read_parquet("cleaned_troop_movements10m.parquet")

In [13]:
#model = pd.read_pickle(r'filepath')


In [9]:
pkl_model = pickle.load(open('proj3model.pkl', 'rb'))

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld


,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
999,2023-06-13 17:16:43,1000,invalid_unit,1.000000,6.000000,1,0,Muunilinst
1999,2023-06-13 17:00:07,2000,invalid_unit,4.000000,8.000000,3,9,Troiken
2999,2023-06-13 16:43:31,3000,invalid_unit,6.000000,3.000000,4,9,Muunilinst
3999,2023-06-13 16:26:55,4000,invalid_unit,2.000000,1.000000,7,3,Dathomir
4999,2023-06-13 16:10:19,5000,invalid_unit,4.000000,6.000000,4,6,Naboo
...,...,...,...,...,...,...,...,...
9995999,2023-02-18 11:32:39,9996000,invalid_unit,1.000000,4.000000,8,0,Muunilinst
9996999,2023-02-18 11:16:04,9997000,invalid_unit,1.000000,9.000000,0,4,Trandosha
9997999,2023-02-18 10:59:28,9998000,invalid_unit,4.000000,5.000000,5,1,Trandosha
9998999,2023-02-18 10:42:52,9999000,invalid_unit,6.000000,8.000000,0,0,Chandrila


In [18]:

dfe = pd.get_dummies(newDataSet, columns=['homeworld', 'unit_type'])

X = dfe.iloc[:, 6:]
newDataSet['predictions'] = pkl_model.predict(X)
display(newDataSet)

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,predictions
0,2023-06-13 17:33:18,1,at-st,2.000000,8.000000,1,1,Glee Anselm,False
1,2023-06-13 17:33:17,2,tie_silencer,4.000000,4.000000,0,1,Trandosha,False
2,2023-06-13 17:33:16,3,at-at,0.000000,3.000000,6,1,Corellia,False
3,2023-06-13 17:33:15,4,tie_silencer,6.000000,1.000000,6,9,Shili,True
4,2023-06-13 17:33:14,5,tie_fighter,0.000000,4.000000,9,6,Muunilinst,False
...,...,...,...,...,...,...,...,...,...
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.000000,6.000000,1,3,Troiken,False
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.000000,6.000000,2,0,Kashyyyk,True
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.000000,7.000000,6,4,Kashyyyk,True
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.000000,6.000000,8,8,Kalee,False
